![ CEA LOGO]
("https://fr.wikipedia.org/wiki/Fichier:CEA_logotype2012.png")




In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# Load data

In [2]:
mydata = np.load('data_ML_Nfreq100_Nbeta20_Nlambda20.npz',allow_pickle=True)
print(mydata.files)

['data', 'freq', 'params']


In [3]:
data = mydata['data']
freq = mydata['freq']
params = mydata['params']

In [4]:
freq.shape

(40000, 256)

In [5]:
data.shape

(40000, 256, 3)

In [6]:
params.shape

(40000, 8)

 ### For pre_processing we are going to use $data$ and $params$

#  I. Using $data$ for pre_processing

####  1. Same Interval

- Nous avons 100 bandes de fréquences $B_i,\forall i \in \overline{1,100}$ .
- Chaque bande $B_i$ à chacune 256 fréquences avec une fréquence min et  max.
- Dans chaque bande $B_i$ , nous avons 400 formes d'ondes donc un total de $N=40 000$ formes d'ondes
  - l'objectif ici est de gérer le problème de chevrauchement entre bande. 
  - Il s'agit simplement d'observer les N formes d'ondes dans une seule $BT$ (bande Totale) 

In [7]:

fr=list(freq[0:40000,:])
frT=np.asarray(fr)
#frTB=frT.flatten()
freqBT=np.sort(frT)
freqBT_unique=np.unique(freqBT)

S_real=np.zeros((40000,freqBT_unique.shape[0],3))
S_imag=np.zeros((40000,freqBT_unique.shape[0],3))
I=np.arange(0,40000,400)
K=np.arange(256,freqBT_unique.shape[0])
J=np.arange(freqBT_unique.shape[0]-256)
for i,j,k in zip(I,J,K):
    S_real[i:(i+400),j:k,:]=np.real(data[i:(i+400),:,:])
    S_imag[i:(i+400),j:k,:]=np.imag(data[i:(i+400),:,:])


In [8]:
freqBT_unique.shape

(2827,)

In [9]:
# mettre toutes les formes d'ondes  sur la même bande de fréquences
def SameInt(S_real,S_imag):
    return np.asarray([u+1j*v for u,v in zip(S_real,S_imag)])

In [10]:
S=SameInt(S_real[:,:,:],S_imag[:,:,:])
S.shape

(40000, 2827, 3)

In [11]:
# verification ( des zéros après la 256 ième fréquence pour FO 0)
S[0,:,0]

array([1.57502989e-26+6.94253168e-27j, 1.56673787e-26+7.13709609e-27j,
       1.55844074e-26+7.33176966e-27j, ...,
       0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
       0.00000000e+00+0.00000000e+00j])

####  2. Normalisation

- $ \overline{S_i}=\frac{S_i}{\rVert S_i \lVert_2},\quad \forall i \in \overline{0,40000}$

In [12]:
# cette fonction normalise seulement une forme d'onde pour un canal "X","Y","Z"
def normalize(S,n,e):
# n nombre de FO, e nombre de canal
    
    return np.asarray([S[n,i,e]/np.linalg.norm(S[n,:,e]) for i in range(S.shape[1])])

# return un tableau contenant chaque forme d'onde normalisée

In [13]:
S1=normalize(S,10,0)
S1

array([-0.0044968 -0.00040616j, -0.00448933-0.00046114j,
       -0.00448187-0.00051608j, ...,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ])

In [14]:
# cette fonction rassemble toutes les formes d'onde normalisée dans un seul tableau 
# n nombre de forme d'onde , e numéro du  canal ("X","Y","Z")
def norm(S):
    
    return (np.asarray([[normalize(S,n,e) for e in range (3)] for n in range(400)])).reshape((-1,2827,3))
# pour les n=400 premières formes d'ondes

In [15]:
S_bar=norm(S)
S_bar[0,:,0]

array([0.0040876 +0.00180176j, 0.00402299+0.00195334j,
       0.00395815+0.00210546j, ..., 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [16]:
S_bar.shape

(400, 2827, 3)

####  3. Génération de v.a uniforme 

- $ \tilde{S_{i,k}}= \mathrm{X}_k \times \overline{S_i},\quad \forall i \in [0,1,...,40000] ,\quad k \in \overline{0,5}\quad avec \quad \mathrm{X}_k \to \cup(1,10)$

In [31]:
# génération de 5 valeurs de loi uniforme
#np.random.seed(0)
nb_fo=400 # nombre de FO utilisable pour l'apprentissage du réseau de neurones (400)
X=[np.random.uniform(1,10,5) for _ in range(nb_fo)]
X[:10,:]

TypeError: list indices must be integers or slices, not tuple

In [18]:
# multiplication de X par chaque FO
def amplitude(S,n):
    
    return np.asarray([S[n,:,:]*i for i in X[n]])

In [19]:
# cette fonction retourne les 5 amplitudes différentes de chaque FO
def amp(S):
     
    return (np.asarray([amplitude(S,n) for n in range(nb_fo)])).reshape((5,-1,2827,3))
# chaque FO retourne 5 sous formes d'onde d'amplitude différentes chacune ayant 2827 fréquences 



In [20]:
S_tilde=amp(S_bar)

In [21]:
S_tilde[:,0,:,0]

array([[ 0.03061405+0.01349429j,  0.03013019+0.01462957j,
         0.02964455+0.01576883j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.01209264+0.00856789j,  0.01178118+0.00901711j,
         0.01146864+0.00946782j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.00800178+0.02548112j,  0.0070644 +0.02578354j,
         0.006124  +0.02608688j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.00945037+0.00725571j, -0.00971962+0.00690862j,
        -0.00998966+0.00656047j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.00140196-0.04225543j,  0.00015515-0.04230941j,
         0.00171658-0.0423635j , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [22]:
S_tilde.shape

(5, 400, 2827, 3)

#  II. Using $params $ for pre_processing

#### 1.Parameters used for learning

-  Amplitude $h_0$
-  Latitude Ecliptic $\beta$
-  Ecliptic Longitude $\lambda$

In [23]:
amplitude=params[:,0] # amplitude
beta=params[:,6] # bêta
lamb=params[:,7] # lambda
#parameters=[]
#zz=np.concatenate((amplitude[0:2],beta[0:2]))
parameters=np.asarray([(i,j,k)for i,j,k in zip(amplitude,beta,lamb)] )

parameters


array([[ 1.00000000e-22,  0.00000000e+00, -3.14159265e+00],
       [ 1.00000000e-22,  0.00000000e+00, -2.81089869e+00],
       [ 1.00000000e-22,  0.00000000e+00, -2.48020473e+00],
       ...,
       [ 1.00000000e-22,  1.57079633e+00,  2.48020473e+00],
       [ 1.00000000e-22,  1.57079633e+00,  2.81089869e+00],
       [ 1.00000000e-22,  1.57079633e+00,  3.14159265e+00]])

In [24]:
parameters.shape

(40000, 3)

 -  $ \overline{h}=[h_i]=\frac{h_0}{\rVert S_i \lVert_2},\forall i \quad\text{allant de 0 à nb_fo}$

 -  $\tilde{h}=[h_i]\times\mathrm{X}_j ,\forall i \in  \overline{0,nb}\quad et\quad j \in\overline{0,5}\quad avec \quad  \mathrm{X} \to \cup(1,10) $

In [27]:
def ParametersAmpli(S,c): # c canal utilisé
    h_bar=[parameters[i,0]/np.linalg.norm(S[i,:,c]) for i in range(nb_fo)]
    h_tilde=np.asarray([[h_bar[j]*i for j in range(nb_fo)] for i in X[j]])
    lam=np.array([[u]*5 for u in lamb ])
    bet=np.array([[v]*5 for v in beta])
    return h_tilde,bet,lam



In [28]:
h_tilde,bet,lam=ParametersAmpli(S,0)

In [29]:
final_params=np.asarray([[(i,j,k)for i,j,k in zip(h_tilde[:,n],bet[n,:],lam[n,:])] for n in range(nb_fo)])
final_params

array([[[143.60419737,   0.        ,  -3.14159265],
        [149.57273892,   0.        ,  -3.14159265],
        [ 57.93428834,   0.        ,  -3.14159265],
        [148.29735468,   0.        ,  -3.14159265],
        [258.31520623,   0.        ,  -3.14159265]],

       [[138.73221391,   0.        ,  -2.81089869],
        [144.49826391,   0.        ,  -2.81089869],
        [ 55.96878246,   0.        ,  -2.81089869],
        [143.26614896,   0.        ,  -2.81089869],
        [249.55148319,   0.        ,  -2.81089869]],

       [[132.31043185,   0.        ,  -2.48020473],
        [137.80957689,   0.        ,  -2.48020473],
        [ 53.37804082,   0.        ,  -2.48020473],
        [136.63449537,   0.        ,  -2.48020473],
        [237.9999827 ,   0.        ,  -2.48020473]],

       ...,

       [[164.80394616,   1.57079633,   2.48020473],
        [171.65360111,   1.57079633,   2.48020473],
        [ 66.48690993,   1.57079633,   2.48020473],
        [170.18993668,   1.57079633,   2.4802

In [30]:
final_data=S_tilde
final_params.shape

(400, 5, 3)